# **Classification**

## Objectives

* Fit and evaluate a classification model to predict if a mushroom is edible or not.

## Inputs

* outputs/datasets/collection/mushrooms.csv
* Instructions on which variables to use for data cleaning and feature engineering. They are found in each respective notebook.

## Outputs

* Train set (features and target)
* Test set (features and target)
* Data cleaning and Feature Engineering pipeline
* Modelling pipeline
* Feature importance plot


---

# Change working directory

* Need to change working directory from the current **jupyter_notebooks** folder to the parent folder in order to access the whole project

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Data

In [ ]:
import pandas as pd

df = pd.read_csv("outputs/datasets/collection/mushrooms.csv")

print(df.shape)
df.head()

---

# ML Pipeline with all data

## ML pipeline for Data Cleaning and Feature Engineering

This pipeline is created from the findings of the Data Cleaning and Feature Engineering notebooks. As there is no Data Cleaning tasks that need to be done, it is just referred to as a Feature Engineering pipeline.

In [ ]:
from sklearn.pipeline import Pipeline
from feature_engine.selection import SmartCorrelatedSelection
from category_encoders import TargetEncoder

def PipelineFeatureEngineering():
    """ 
    Creates feature engineering pipeline

    Args:
        None
    
    Returns:
        pipeline_base: a sklearn pipeline object, containing all the feature
                        engineering steps required
    """

    pipeline_base = Pipeline([
        ("TargetCategoricalEncoder", TargetEncoder()),
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, 
                                                            method="spearman", 
                                                            threshold=0.6, 
                                                            selection_method="variance"))
    ])

    return pipeline_base

PipelineFeatureEngineering()

## ML Pipeline for Modelling and Hyperparameter Optimisation

This pipeline will have a Standard Scaler as its first step, which transforms the variables into their z scores. The most relevant features to the model will then be selected from the set of variables, and then finally fit by the model, which will now be chosen based on a model hyperparameter grid search.

In [ ]:
# Feat Scaling
from sklearn.preprocessing import StandardScaler

# Feat Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier


def PipelineClf(model):
    """ 
    Creates classifier model pipeline
    Taken from Walkthrough Project 02 - Churnometer

    Args:
        model: the desired sklearn model to use in the pipeline

    Returns:
        pipeline_base: a sklearn pipeline object containing the feature scaling
                        feature selection and model steps
    """
    pipeline_base = Pipeline([
        ("scaler", StandardScaler()),
        ("feat_selection", SelectFromModel(model)),
        ("model", model),
    ])

    return pipeline_base

Custom Class for Hyperparameter Optimisation - this class, taken from the model cross validation lessons, will allow performing a hyperparameter grid search to determine the best performing classifier model and it's ideal hyperparameters for the model pipeline.

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

class HyperparameterOptimizationSearch:
    """ 
    Creates a set of models to iterate over and stores their performances to assess the best one
    Taken from Walkthrough Project 02 - Churnometer 

    Attributes:
        models: a dictionary of the different models to search over
        params: a dictionary of the hyperparameters for each model to search over
    
    Methods:
        fit - fits each of the declared models for each of their stated hyperparameter sets
        score_summary - generates a summary output of each fitted model's performance for comparison purposes
    """
    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None):
        """ 
        Fits each of the search's models using a gridsearch 
        on each set of hyperparameters input for each model
        Taken from Walkthrough Project 02 - Churnometer 

        Args:
            X: the feature train data for fitting
            y: the target train data for fitting
            cv: the number of folds in the cross validation for each fit
            n_jobs: number of processor cores to use in each fit
            verbose: controls whether to give verbose output of fitting

        Returns:
            None
        """
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")

            model = PipelineClf(self.models[key])
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, )
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        """ 
        Creates a summation of each models performance and stores it in a DataFrame
        Taken from Walkthrough Project 02 - Churnometer 

        Args:
            sort_by: the parameter the models in the DataFrame will be ordered by

        Returns:
            df[columns]:
            self.grid_searches: a dictionary of grid search objects, which can be parsed
                                to find the best hyperparameters for each model
        """
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        return df[columns], self.grid_searches

## Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['edible'], axis=1),
    df['edible'],
    test_size=0.2,
    random_state=0,
)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

## Fit and Transform Train and Test Sets

Using `y_train` as the target input for the `"TargetEncoder"` step, the feature engineering pipeline is fit to and then transforms the Train Set, and then is used to transform the Test Set

In [ ]:
pipeline_data_cleaning_feat_eng = PipelineFeatureEngineering()
X_train = pipeline_data_cleaning_feat_eng.fit_transform(X_train, y_train)
X_test = pipeline_data_cleaning_feat_eng.transform(X_test)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

---

# Handle Target Imbalance

Check Train Set edibility distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
y_train.value_counts().plot(kind='bar', title='Train Set Target Distribution')
plt.show()

It appears the dataset is slightly skewed toward having more edible mushrooms. To balance the target, we use SMOTE (Synthetic Minority Oversampling TEchnique) on the Train Set

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(sampling_strategy='minority', random_state=0)
X_train, y_train = oversample.fit_resample(X_train, y_train)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Check Train Set Target distribution after resampling - it is now balanced between edible and poisonous mushrooms.

In [ ]:
import matplotlib.pyplot as plt
y_train.value_counts().plot(kind='bar', title='Train Set Target Distribution')
plt.show()

---

# Grid Search CV - Sklearn

### Use standard hyperparameters to find most suitable algorithm

We define a set of different classifier algorithms to perform a grid search over. The hyperparameters will be left to their defaults, here we wish to find the best algorithm only.

In [ ]:
models_quick_search = {
    "LogisticRegression": LogisticRegression(random_state=0),
    "XGBClassifier": XGBClassifier(random_state=0),
    "DecisionTreeClassifier": DecisionTreeClassifier(random_state=0),
    "RandomForestClassifier": RandomForestClassifier(random_state=0),
    "GradientBoostingClassifier": GradientBoostingClassifier(random_state=0),
    "ExtraTreesClassifier": ExtraTreesClassifier(random_state=0),
    "AdaBoostClassifier": AdaBoostClassifier(random_state=0),
}

params_quick_search = {
    "LogisticRegression": {},
    "XGBClassifier": {},
    "DecisionTreeClassifier": {},
    "RandomForestClassifier": {},
    "GradientBoostingClassifier": {},
    "ExtraTreesClassifier": {},
    "AdaBoostClassifier": {},
}

Quick GridSearch CV - Binary Classifier. As we need to optimize with respect to the precision and recall on edibility, we will set the scorer to f1_score, a metric that includes the two. 

In [ ]:
from sklearn.metrics import make_scorer, f1_score
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train,
           scoring =  make_scorer(f1_score, pos_label=1),
           n_jobs=-1, cv=5)

Check results - will select `AdaBoostClassifier` as best performing model.

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary 

Define a set of hyperparameters for `AdaBoostClassifier` to perform the grid search over.

In [ ]:
models_search = {
    "AdaBoostClassifier":AdaBoostClassifier(random_state=0),
}

params_search = {
    "AdaBoostClassifier":{
        'model__n_estimators': [10, 25, 50, 100, 500],
        'model__learning_rate': [0.01, 0.05, 0.2, 1],
        'model__base_estimator': [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=3), DecisionTreeClassifier(max_depth=5)]
    }
}

Extensive GridSearch CV - Binary Classifier. As we need to optimize with respect to the precision and recall on edibility, we will set the scorer to f1_score, a metric that includes the two. 

In [ ]:
from sklearn.metrics import make_scorer, f1_score
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train,
           scoring = make_scorer(f1_score, pos_label=1),
           n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary 

Get best model name

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

Get parameters for best model

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

Define the best clf pipeline by pulling it from the `grid_search_pipelines` dictionary

In [ ]:
pipeline_clf = grid_search_pipelines[best_model].best_estimator_
pipeline_clf

## Assess feature importance

In [ ]:
X_train.head()

Using current model, can assess feature importance from model `.feature_importances_`

In [ ]:
df_feature_importance = (pd.DataFrame(data={
    'Feature': X_train.columns[pipeline_clf['feat_selection'].get_support()],
    'Importance': pipeline_clf['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

best_features = df_feature_importance['Feature'].to_list()

print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

## Evaluate Pipeline on Train and Test Sets

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def confusion_matrix_and_report(X, y, pipeline, label_map):
    """ 
    Displays the confusion matrix and classification report for a given model
    Taken from Walkthrough Project 02 - Churnometer

    Args:
        X: the feature data
        y: the target data
        pipeline: the model pipeline to assess
        label_map: a dictionary to map classifier output to their physical meanings

    Returns:
        None 
    """
    prediction = pipeline.predict(X)

    print('---  Confusion Matrix  ---')
    print(pd.DataFrame(confusion_matrix(y_true=prediction, y_pred=y),
            columns=[["Actual " + sub for sub in label_map]],
            index=[["Prediction " + sub for sub in label_map]]
          ))
    print("\n")

    print('---  Classification Report  ---')
    print(classification_report(y, prediction, target_names=label_map), "\n")


def clf_performance(X_train, y_train, X_test, y_test, pipeline, label_map):
    """ 
    Calls confusion_matrix_and_report for the train and test set to display
    classifier performance
    Taken from Walkthrough Project 02 - Churnometer
    
    Args:
        X_train: the training feature data
        y_train: the training target data
        X_test: the testing feature data
        y_test: the testing target data
        pipeline: the model pipeline to assess the performance of 
        label_map: a dictionary to map classifier output to their physical meanings

    Returns:
        None
    """
    print("#### Train Set #### \n")
    confusion_matrix_and_report(X_train, y_train, pipeline, label_map)

    print("#### Test Set ####\n")
    confusion_matrix_and_report(X_test, y_test, pipeline, label_map)

Evaluation: We cross check with metrics defined in the ML business case

* A precision of 1 on predicting whether a mushroom is edible
* A recall of at least 0.9 on predicting if a mushroom is edible

In [ ]:
clf_performance(X_train=X_train, y_train=y_train,
                X_test=X_test, y_test=y_test,
                pipeline=pipeline_clf,
                label_map=['poisonous', 'edible']
                )

The model performs at the required level of precision and recall on the test set, and therefore is useable for the business case.

---

## Push files to Repo

Will generate following files
* Train set
* Test set
* Data cleaning and Feature Engineering pipeline
* Modeling pipeline
* Features importance plot

In [ ]:
import joblib
import os

version = 'v1'
file_path = f'outputs/ml_pipeline/predict_edible/{version}'

try:
    os.makedirs(name=file_path)
except Exception as e:
    print(e)

## Train Set

Variables are already transformed, and SMOTE has been applied

In [ ]:
print(X_train.shape)
X_train.head()

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)

In [ ]:
y_train

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)

## Test Set

Variables are already transformed

In [ ]:
print(X_test.shape)
X_test.head()

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)

In [ ]:
y_test

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

## ML Pipelines: Data Cleaning and Feat Eng pipeline and Modelling Pipeline

Will save 2 pipelines:
* Both will be used sequentially to predict Live Data.
* To predict on Train Set/Test Set, only use pipeline_clf since the data is already processed.

Pipeline for Data Cleaning and Feature Engineering

In [ ]:
pipeline_data_cleaning_feat_eng

In [ ]:
joblib.dump(value=pipeline_data_cleaning_feat_eng ,
            filename=f"{file_path}/clf_pipeline_feat_eng.pkl")

Pipeline for feature scaling and model

In [ ]:
pipeline_clf

In [ ]:
joblib.dump(value=pipeline_clf ,
            filename=f"{file_path}/clf_pipeline_model.pkl")

## Feature Importance plot

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')